In [25]:
import pandas as pd

In [26]:
df = pd.read_csv("../data/raw_data.csv")
df.head()

,customer_id,customer_age,customer_income,home_ownership,employment_duration,loan_intent,loan_grade,loan_amnt,loan_int_rate,term_years,historical_default,cred_hist_length,Current_loan_status
0,1.0,22,59000,RENT,123.0,PERSONAL,C,"£35,000.00",16.02,10,Y,3,DEFAULT
1,2.0,21,9600,OWN,5.0,EDUCATION,A,"£1,000.00",11.14,1,NaN,2,NO DEFAULT
2,3.0,25,9600,MORTGAGE,1.0,MEDICAL,B,"£5,500.00",12.87,5,N,3,DEFAULT
3,4.0,23,65500,RENT,4.0,MEDICAL,B,"£35,000.00",15.23,10,N,2,DEFAULT
4,5.0,24,54400,RENT,8.0,MEDICAL,B,"£35,000.00",14.27,10,Y,4,DEFAULT


In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32586 entries, 0 to 32585
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   customer_id          32583 non-null  float64
 1   customer_age         32586 non-null  int64  
 2   customer_income      32586 non-null  object 
 3   home_ownership       32586 non-null  object 
 4   employment_duration  31691 non-null  float64
 5   loan_intent          32586 non-null  object 
 6   loan_grade           32586 non-null  object 
 7   loan_amnt            32585 non-null  object 
 8   loan_int_rate        29470 non-null  float64
 9   term_years           32586 non-null  int64  
 10  historical_default   11849 non-null  object 
 11  cred_hist_length     32586 non-null  int64  
 12  Current_loan_status  32582 non-null  object 
dtypes: float64(3), int64(3), object(7)
memory usage: 3.2+ MB


In [28]:
df.describe(include='all')


,customer_id,customer_age,customer_income,home_ownership,employment_duration,loan_intent,loan_grade,loan_amnt,loan_int_rate,term_years,historical_default,cred_hist_length,Current_loan_status
count,32583.000000,32586.000000,32586,32586,31691.000000,32586,32586,32585,29470.000000,32586.000000,11849,32586.000000,32582
unique,NaN,NaN,4299,4,NaN,6,5,755,NaN,NaN,2,NaN,2
top,NaN,NaN,60000,RENT,NaN,EDUCATION,A,"£10,000.00",NaN,NaN,Y,NaN,NO DEFAULT
freq,NaN,NaN,1046,16451,NaN,6454,15661,2664,NaN,NaN,6128,NaN,25742
mean,16289.497806,27.732769,NaN,NaN,4.790161,NaN,NaN,NaN,11.011553,4.761738,NaN,5.804026,NaN
std,9405.919628,6.360528,NaN,NaN,4.142746,NaN,NaN,NaN,3.240440,2.471107,NaN,4.055078,NaN
min,1.000000,3.000000,NaN,NaN,0.000000,NaN,NaN,NaN,5.420000,1.000000,NaN,2.000000,NaN
25%,8144.500000,23.000000,NaN,NaN,2.000000,NaN,NaN,NaN,7.900000,3.000000,NaN,3.000000,NaN
50%,16288.000000,26.000000,NaN,NaN,4.000000,NaN,NaN,NaN,10.990000,4.000000,NaN,4.000000,NaN
75%,24433.500000,30.000000,NaN,NaN,7.000000,NaN,NaN,NaN,13.470000,7.000000,NaN,8.000000,NaN


In [29]:
df["Current_loan_status"].value_counts()

Current_loan_status
NO DEFAULT    25742
DEFAULT        6840
Name: count, dtype: int64

In [30]:
df["recovery_success"] = df["Current_loan_status"].apply(
    lambda x: 1 if x== 'NO DEFAULT' else 0
)
df["recovery_success"].value_counts()

recovery_success
1    25742
0     6844
Name: count, dtype: int64

In [31]:
# Clean loan_amnt: remove currency symbols and commas, convert to float
df['loan_amnt'] = (
    df['loan_amnt']
    .astype(str)
    .str.replace('£', '', regex=False)
    .str.replace(',', '', regex=False)
    .astype(float)
)


In [32]:
# Clean customer_income: remove commas and convert to float
df['customer_income'] = (
    df['customer_income']
    .astype(str)
    .str.replace(',', '', regex=False)
    .astype(float)
)


In [33]:
feature_cols = [
    "customer_age",
    "customer_income",
    "home_ownership",
    "employment_duration",
    "loan_intent",
    "loan_grade",
    "loan_amnt",
    "loan_int_rate",
    "term_years",
    "historical_default",
    "cred_hist_length"
]
X = df[feature_cols]
y = df["recovery_success"]


In [34]:
X.isnull().sum()

customer_age               0
customer_income            0
home_ownership             0
employment_duration      895
loan_intent                0
loan_grade                 0
loan_amnt                  1
loan_int_rate           3116
term_years                 0
historical_default     20737
cred_hist_length           0
dtype: int64

In [35]:
num_cols = X.select_dtypes(include=["int64","float64"]).columns
cat_cols = X.select_dtypes(include=["object"]).columns

X[num_cols] = X[num_cols].fillna(X[num_cols].median())
X[cat_cols] = X[cat_cols].fillna('Unknown')

C:\Users\dell\AppData\Local\Temp\ipykernel_11188\3866061330.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[num_cols] = X[num_cols].fillna(X[num_cols].median())
C:\Users\dell\AppData\Local\Temp\ipykernel_11188\3866061330.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[cat_cols] = X[cat_cols].fillna('Unknown')


In [36]:
X.isnull().sum()

customer_age           0
customer_income        0
home_ownership         0
employment_duration    0
loan_intent            0
loan_grade             0
loan_amnt              0
loan_int_rate          0
term_years             0
historical_default     0
cred_hist_length       0
dtype: int64

In [37]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [38]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), cat_cols)
    ]
)
preprocessor

ColumnTransformer(transformers=[('num', StandardScaler(),
                                 Index(['customer_age', 'customer_income', 'employment_duration', 'loan_amnt',
       'loan_int_rate', 'term_years', 'cred_hist_length'],
      dtype='object')),
                                ('cat', OneHotEncoder(handle_unknown='ignore'),
                                 Index(['home_ownership', 'loan_intent', 'loan_grade', 'historical_default'], dtype='object'))])

In [39]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


In [40]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score


In [41]:
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(max_iter=1000))
])
model

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  Index(['customer_age', 'customer_income', 'employment_duration', 'loan_amnt',
       'loan_int_rate', 'term_years', 'cred_hist_length'],
      dtype='object')),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  Index(['home_ownership', 'loan_intent', 'loan_grade', 'historical_default'], dtype='object'))])),
                ('classifier', LogisticRegression(max_iter=1000))])

In [42]:
# Train model
model.fit(X_train, y_train)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  Index(['customer_age', 'customer_income', 'employment_duration', 'loan_amnt',
       'loan_int_rate', 'term_years', 'cred_hist_length'],
      dtype='object')),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  Index(['home_ownership', 'loan_intent', 'loan_grade', 'historical_default'], dtype='object'))])),
                ('classifier', LogisticRegression(max_iter=1000))])

In [43]:
# Predictions
y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)[:, 1]

# Metrics
print("Accuracy:", accuracy_score(y_test, y_pred))
print("ROC AUC:", roc_auc_score(y_test, y_proba))


Accuracy: 0.9436943847806075
ROC AUC: 0.9837448845443051


In [44]:
import joblib

# Save trained pipeline
joblib.dump(model, "../models/recovery_model.pkl")


['../models/recovery_model.pkl']

In [45]:
# Define DCA profiles (simulated)
dca_profiles = {
    "DCA_A": {
        "strength": "high_value",
        "effectiveness": 0.90
    },
    "DCA_B": {
        "strength": "medium_value",
        "effectiveness": 0.80
    },
    "DCA_C": {
        "strength": "low_value",
        "effectiveness": 0.70
    }
}


In [46]:
def recommend_dca(recovery_probability, outstanding_amount):
    scores = {}
    
    for dca, profile in dca_profiles.items():
        score = recovery_probability * profile["effectiveness"]
        
        # Slight bias based on case size
        if outstanding_amount > 50000 and profile["strength"] == "high_value":
            score *= 1.1
        elif outstanding_amount <= 50000 and profile["strength"] == "low_value":
            score *= 1.05
        
        scores[dca] = score
    
    best_dca = max(scores, key=scores.get)
    return best_dca, scores


In [47]:
# Take one test sample
sample = X_test.iloc[[0]]

# Predict recovery probability
recovery_prob = model.predict_proba(sample)[0][1]

# Get outstanding amount
amount = sample['loan_amnt'].values[0]

# Recommend DCA
best_dca, dca_scores = recommend_dca(recovery_prob, amount)

print("Recovery Probability:", recovery_prob)
print("Recommended DCA:", best_dca)
print("All DCA Scores:", dca_scores)


Recovery Probability: 0.3840450999903664
Recommended DCA: DCA_A
All DCA Scores: {'DCA_A': 0.3456405899913298, 'DCA_B': 0.30723607999229313, 'DCA_C': 0.2822731484929193}


In [48]:
# Get categorical feature names
cat_features = preprocessor.transformers_[1][2]

# Get categories learned by OneHotEncoder
encoder = model.named_steps['preprocessor'].transformers_[1][1]

for feature, cats in zip(cat_features, encoder.categories_):
    print(feature, ":", list(cats))


home_ownership : ['MORTGAGE', 'OTHER', 'OWN', 'RENT']
loan_intent : ['DEBTCONSOLIDATION', 'EDUCATION', 'HOMEIMPROVEMENT', 'MEDICAL', 'PERSONAL', 'VENTURE']
loan_grade : ['A', 'B', 'C', 'D', 'E']
historical_default : ['N', 'Unknown', 'Y']
